# Two-Class Support Vector Machine exercise

Complete and hand in this completed worksheet with your assignment submission.

In this exercise you will:
    
- Understand the logic of the following code to use SVM for image classification.
- Implement a linear SVM by calling functions from scikit-learn module.
- Tune parameters of SVM. Analyze the tuned results.

In [14]:
!wget http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz -O cifar-10-python.tar.gz # 下载文件
!tar -xzvf cifar-10-python.tar.gz # 解压文件
!rm cifar-10-python.tar.gz

--2024-03-28 06:51:46--  http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘cifar-10-python.tar.gz’

cifar-10-python.tar 100%[===================>] 162.60M  52.2MB/s    in 3.3s    

2024-03-28 06:51:49 (50.0 MB/s) - ‘cifar-10-python.tar.gz’ saved [170498071/170498071]

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1


### (1) Prepare two-class data
Prepare CIFAR-10 images. Note that we only use two classes (cat, dog) here for image classification.
 - training: 100 images of cats and dogs
 - testing: as well as 100 images

In [15]:
# define functions for loading CIFAR-10.

from six.moves import cPickle as pickle
import numpy as np
import os
import platform

def load_pickle(f):
    version = platform.python_version_tuple() # python版本是2. or 3.
    if version[0] == '2':
        return  pickle.load(f)
    elif version[0] == '3':
        return  pickle.load(f, encoding='latin1')
    raise ValueError("invalid python version: {}".format(version))

def load_CIFAR_batch(filename):
    """ load single batch of cifar """
    with open(filename, 'rb') as f:
        datadict = load_pickle(f)
        X = datadict['data']
        Y = datadict['labels']
        X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
        # batch操作，把data转换成特定的形状组合就好啦 -- sklearn处理矩阵数据
        Y = np.array(Y)
        return X, Y

def load_CIFAR10(ROOT):
    """ load all of cifar """
    xs = []
    ys = []
    for b in range(1,6):
        f = os.path.join(ROOT, 'data_batch_%d' % (b, ))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    Xte, Yte = load_CIFAR_batch(os.path.join(ROOT, 'test_batch'))
    return Xtr, Ytr, Xte, Yte

In [16]:
# Load the raw CIFAR-10 data.
cifar10_dir = './cifar-10-batches-py'

# [**********] Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
try:
   del X_train, y_train
   del X_test, y_test
   print('Clear previously loaded data.')
except:
   pass

X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

# As a sanity check, we print out the size of the training and test data.
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Clear previously loaded data.
Training data shape:  (50000, 32, 32, 3)
Training labels shape:  (50000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [17]:
# Extract the two class images. Separate the data into training and validation set.

train_smaple_num = 100
test_smaple_num = 100

# label of 'cat' and 'dog'
selected_label = [3, 5]

# 【Train】
# Re-split the 'cat' and 'dog' samples
# 生成一个和y_train形状一样、true or false的矩阵，用来mask
cat_mask_train = y_train == selected_label[0]
dog_mask_train = y_train == selected_label[1]
X2_train_p1 = X_train[cat_mask_train][:train_smaple_num] # cat
y2_train_p1 = np.zeros(X2_train_p1.shape[0], dtype=np.int64) # label=0 -- 和sklearn中期望的svm label统一
# 【实际上svm应该是1、-1的标注】
X2_train_p2 = X_train[dog_mask_train][:train_smaple_num] # dog
y2_train_p2 = np.ones(X2_train_p2.shape[0], dtype=np.int64) # label=1
X2_train = np.concatenate([X2_train_p1, X2_train_p2], axis=0)
y2_train = np.concatenate([y2_train_p1, y2_train_p2], axis=0)

# 【Test】
cat_mask_test = y_test == selected_label[0]
dog_mask_test = y_test == selected_label[1]
X2_test_p1 = X_test[cat_mask_test][:test_smaple_num]
y2_test_p1 = np.zeros(X2_test_p1.shape[0], dtype=np.int64)
X2_test_p2 = X_test[dog_mask_test][:test_smaple_num]
y2_test_p2 = np.ones(X2_test_p2.shape[0], dtype=np.int64)
X2_test = np.concatenate([X2_test_p1, X2_test_p2], axis=0)
y2_test = np.concatenate([y2_test_p1, y2_test_p2], axis=0)

# Double check the size of the training and test data.
print('Training data shape: ', X2_train.shape)
print('Training labels shape: ', y2_train.shape)
print('Test data shape: ', X2_test.shape)
print('Test labels shape: ', y2_test.shape)

Training data shape:  (200, 32, 32, 3)
Training labels shape:  (200,)
Test data shape:  (200, 32, 32, 3)
Test labels shape:  (200,)


### (2) Extract feature vectors
We utilized a pretrained CNN to extract the features vectors. Please understand the logic of following code.

In [18]:
# Extract the feature vectors of images.
# We utilized a pretrained ResNet18 network to extract the layer4 features as the sample features.

import torch
import torch.nn as nn
from torchvision import models

class ResBase18(nn.Module):
    def __init__(self):
        super(ResBase18, self).__init__()
        # model_resnet18 = models.resnet18(pretrained=True)
        # 预训练、含残差连接的神经网络
        model_resnet18 = models.resnet18(pretrained=True)
        self.conv1 = model_resnet18.conv1
        self.bn1 = model_resnet18.bn1
        self.relu = model_resnet18.relu
        self.maxpool = model_resnet18.maxpool
        self.layer1 = model_resnet18.layer1
        self.layer2 = model_resnet18.layer2
        self.layer3 = model_resnet18.layer3
        self.layer4 = model_resnet18.layer4
        self.avgpool = model_resnet18.avgpool

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        return x

# define a instance of network, turn to evaluation mode.
model = ResBase18()
model.eval()

# extract features of training and test samples.
"""
In PyTorch, the `permute()` function is used to [[perform dimension rearrangement on a tensor.]]
It allows for swapping the order of axes or reordering the dimensions of a tensor.

In the given code, `permute(0, 3, 1, 2)` is used to change the dimension order of the `X2_train` tensor
[[from `(batch_size, height, width, channels)` to `(batch_size, channels, height, width)`]].
This is because in PyTorch, the expected input tensor format for convolutional neural network models
is usually `(batch_size, channels, height, width)`.
By using the `permute()` function to rearrange the dimensions of the tensor,
the data is transformed into a tensor format that is suitable for model input.
"""
with torch.no_grad():
    X2_train_feat = model(torch.FloatTensor(X2_train).permute(0, 3, 1, 2)) # rearrange the dimensions to suit model input
    X2_test_feat = model(torch.FloatTensor(X2_test).permute(0, 3, 1, 2))
X2_train_feat = X2_train_feat.numpy()
X2_test_feat = X2_test_feat.numpy()

# Check the dimension of extracted features
print('Training feature shape: ', X2_train_feat.shape)
print('Testing feature shape: ', X2_test_feat.shape)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training feature shape:  (200, 512)
Testing feature shape:  (200, 512)


### (3) Implement a Linear Support Vector Machine(SVM)
Check the documentation at https://scikit-learn.org/stable/modules/classes.html#module-sklearn.svm to see how to use these functions to train and test a Support Vector Machine(SVM).  
**TODO**:
- 1) implement a linear SVM;
- 2) make predictions with your SVM.

In [19]:
from sklearn.svm import SVC

########################################################################################################
# Step 1 TODO:
# Training a linear SVM using training set.
# You need to write code that trains a SVM. Please refer to the document of sklearn.svm.
########################################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# Create a svm instance
svc = SVC(kernel="linear")

# Train the svm instance svc with the training set samples 'X2_train_feat' and labels 'y2_train' as input
trained_svm = svc.fit(X2_train_feat, y2_train)

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****





########################################################################################################
# Step 2 TODO:
# Using the trained SVM to predict the classification results of validation set.
# You need to write code that test the SVM you implemented above.
# Again you may refer to the document to see if any functions are already defined for prediction.
########################################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# Test our trained svm instance svc with the test set samples 'X2_test_feat'
y2_pred = trained_svm.predict(X2_test_feat)


# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

# Print the predicted outputs and accuracies.
# We use precision, recall and f-measure to measure the accuracy of image classification.
# To understand the meaning of these metrics, you can refer to
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html.


#### Tune paramaters and analyze the results
**TODO**: Document the parameters you have tried such as the kernel function or the penalty dunction, and write down your analysis.

In [20]:

########################################################################################################
# Step 3 TODO:                                                                                         #
# You may tune parameters like kernels and penality functions to improve your testing results.
# Then, you need to write down what parameters you have tried and the analysis of results.
# Note that you can write your analysis in the next block or in the writing assignment.
########################################################################################################
from sklearn.metrics import classification_report, confusion_matrix

kernel_list = ["linear", "poly", "rbf", "sigmoid"]
C_list = [0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 4, 6]
penalty_list = ["l2", "l1"]

for cur_kernel in kernel_list:
  for cur_C in C_list:
    for cur_penalty in penalty_list:
      print("current parameter: Kernel={0}, C={1}, Penalty={2}".format(cur_kernel,cur_C,cur_penalty))
      cur_svc = SVC(kernel=cur_kernel, C=cur_C, cache_size = 500)
      trained_svm = cur_svc.fit(X2_train_feat, y2_train)
      y2_pred = trained_svm.predict(X2_test_feat)
      print(classification_report(y2_test,y2_pred))
      print("\n")



current parameter: Kernel=linear, C=0.5, Penalty=l2
              precision    recall  f1-score   support

           0       0.59      0.53      0.56       100
           1       0.57      0.63      0.60       100

    accuracy                           0.58       200
   macro avg       0.58      0.58      0.58       200
weighted avg       0.58      0.58      0.58       200



current parameter: Kernel=linear, C=0.5, Penalty=l1
              precision    recall  f1-score   support

           0       0.59      0.53      0.56       100
           1       0.57      0.63      0.60       100

    accuracy                           0.58       200
   macro avg       0.58      0.58      0.58       200
weighted avg       0.58      0.58      0.58       200



current parameter: Kernel=linear, C=0.75, Penalty=l2
              precision    recall  f1-score   support

           0       0.61      0.57      0.59       100
           1       0.60      0.64      0.62       100

    accuracy         

**Analysis**:
1. **Experiment result**:

Best parameters: Kernel=linear, C=0.75, Penalty=l2/l1 (regularization l1 and l2 have the same metics value.)

|  | precision | recall | f1-score | support |
| --- | --- | --- | | |
| 0 | 0.61 | 0.57 |0.59|100|
| 1 | 0.60 | 0.64 |0.62|100|
| accuracy | - | - |0.60|200|
| macro avg | 0.61 | 0.60 |0.60|200|
| weighted avg | 0.61 | 0.60 |0.60|200|


2. **Analysis**:
- Initial feature is likely linear separable space? (probably)
- Given abundant computing time and resources, SVM possesses a strong fitting capacity. However, in this experiment, we encountered not a perfect result. There are some resonable analysis below:

As we known, SVM fits a linear decisionn boundary in initial vector space.
Meanwhile, it is proved that when we map initial vector space to a high dimentional space (dimension might be infinite), mapped data is linearly separable for sure.


Moreover, The kernel function and the mapping function have a one-to-one correspondence, and the feature space induced by a kernel function is called a reproducing Hilbert space.

That we are constricted by finite computing resources and time could be significant reason that we could not attempt lots of high dimention vector space (the same as lots of mapping or kernel function) to test whether mapped data in this space is linearly separable, generating a not perfect experiment reslt.


